# [Module 3.2] Personalize 솔류션 버전 평가 지표 확인하기

이 노트북은 아래와 같은 작업을 합니다.

* 솔류션 버전으로 부터 성능 지표 구하기
* 성능 지표 분석

---
노트북의 실행 소요 시간은 약 1분 입니다.

In [6]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [7]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [8]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

## 솔루션 버전 평가를 위한 학습 데이터 분리 방법

이번 파트에서는 Amazon Personalize에서 기본으로 제공하는 솔루션에 대한 평가 지표를 확인해 봅니다. 
Amazon Personalize에서는 평가 지표를 생성하기 위해 약 랜덤으로 10% 사용자의 interaction data를 테스트 용으로 활용합니다. 

아래 이미지는 Amazon Personalize가 데이터를 분리하는 방법을 보여줍니다. 사용자가 10 명이고 각각 10 개의 상호 작용이있는 경우 (여기에서 원은 Interaction data를 나타냄) 타임 스탬프를 기준으로 가장 오래된 것부터 최신 것까지 나열된 것입니다. Amazon Personalize는 사용자의 90 % (파란색 원)의 모든 Interaction 데이터를 사용하여 솔루션 버전을 훈련시키고 나머지 10 %는 평가를 위해 사용합니다. 나머지 10 %의 각 사용자에 대해 Interaction data (녹색 원)의 90 %가 훈련 된 모델의 입력값으로 사용됩니다. 데이터의 나머지 10 % (주황색 원)는 모델에서 생성 된 추천 결과물과 비교되고 평가 지표를 계산하는 데 사용됩니다.



![personalize metrics](static/imgs/personalize_metrics.png)



#### 솔류션 버전의 성능 지표 구하기



In [9]:
metrics=[]

def build_metric_matrix(solution,response):
    metrics.append([solution,
                response['metrics']['coverage'],
                response['metrics']['mean_reciprocal_rank_at_25'],
                response['metrics']['normalized_discounted_cumulative_gain_at_5'],
                response['metrics']['normalized_discounted_cumulative_gain_at_10'],
                response['metrics']['normalized_discounted_cumulative_gain_at_25'],
                response['metrics']['precision_at_5'],
                response['metrics']['precision_at_10'],
                response['metrics']['precision_at_25']])

#### Metrics: Popularity

아래는 생성한 7개의 솔류션 버전에 대해서 성능 지표를 얻습니다.

In [10]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = popularity_solution_version_arn 
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('popularity',get_solution_metrics_response)


{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-42535/ec593d6a",
  "metrics": {
    "coverage": 0.0136,
    "mean_reciprocal_rank_at_25": 0.0559,
    "normalized_discounted_cumulative_gain_at_10": 0.067,
    "normalized_discounted_cumulative_gain_at_25": 0.1112,
    "normalized_discounted_cumulative_gain_at_5": 0.0395,
    "precision_at_10": 0.02,
    "precision_at_25": 0.0182,
    "precision_at_5": 0.0163
  },
  "ResponseMetadata": {
    "RequestId": "5d70d4b7-4967-4708-9fc5-9c72ddb51be7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 15 Jun 2022 03:47:29 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "412",
      "connection": "keep-alive",
      "x-amzn-requestid": "5d70d4b7-4967-4708-9fc5-9c72ddb51be7"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: User-Personalization

In [11]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

build_metric_matrix('user_personalization',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-42535/1cdb2182",
  "metrics": {
    "coverage": 0.2973,
    "mean_reciprocal_rank_at_25": 0.2223,
    "normalized_discounted_cumulative_gain_at_10": 0.2385,
    "normalized_discounted_cumulative_gain_at_25": 0.3205,
    "normalized_discounted_cumulative_gain_at_5": 0.182,
    "precision_at_10": 0.0874,
    "precision_at_25": 0.0639,
    "precision_at_5": 0.0971
  },
  "ResponseMetadata": {
    "RequestId": "5859d31b-2333-492b-b740-fd62fe78c2fb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 15 Jun 2022 03:47:34 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "424",
      "connection": "keep-alive",
      "x-amzn-requestid": "5859d31b-2333-492b-b740-fd62fe78c2fb"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: SIMS

In [12]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('sims',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-sims-42535/f1b2522f",
  "metrics": {
    "coverage": 0.7622,
    "mean_reciprocal_rank_at_25": 0.1802,
    "normalized_discounted_cumulative_gain_at_10": 0.1837,
    "normalized_discounted_cumulative_gain_at_25": 0.2524,
    "normalized_discounted_cumulative_gain_at_5": 0.1519,
    "precision_at_10": 0.0533,
    "precision_at_25": 0.0404,
    "precision_at_5": 0.0689
  },
  "ResponseMetadata": {
    "RequestId": "40964504-6e85-40fc-b2a1-4c708147ba68",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 15 Jun 2022 03:47:34 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "409",
      "connection": "keep-alive",
      "x-amzn-requestid": "40964504-6e85-40fc-b2a1-4c708147ba68"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: Ranking

In [13]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = ranking_solution_version_arn 
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('ranking',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-42535/7ff102dc",
  "metrics": {
    "coverage": 0.2951,
    "mean_reciprocal_rank_at_25": 0.2341,
    "normalized_discounted_cumulative_gain_at_10": 0.2454,
    "normalized_discounted_cumulative_gain_at_25": 0.3309,
    "normalized_discounted_cumulative_gain_at_5": 0.193,
    "precision_at_10": 0.0822,
    "precision_at_25": 0.0622,
    "precision_at_5": 0.0981
  },
  "ResponseMetadata": {
    "RequestId": "f5fdfad5-1372-4621-a9c8-69bc4872d36c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 15 Jun 2022 03:47:35 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "411",
      "connection": "keep-alive",
      "x-amzn-requestid": "f5fdfad5-1372-4621-a9c8-69bc4872d36c"
    },
    "RetryAttempts": 0
  }
}


## Summary Metrics

#### 1. 솔류션 평가 지표 정의
[솔류션 평가 지표 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)
는 개발자 문서의 링크 참조 바랍니다. 또한 이 링크 [솔류션 평가 정의 예제](http://francescopochetti.com/recommend-expedia-hotels-with-amazon-personalize-the-magic-of-hierarchical-rnns/) 의 페이지 맨 아래 쪽을 보시면 조금 더 직관적인 그림을 보실 수 있습니다.
 <br>
또한 reciprocal_rank_at_5, normalized_discounted_cumulative_gain_at_5,precision_at_5 의 예제는 아래와 같습니다. 
* Exmaple
    * 5 개의 추천리스트를 제공했고, 이 중에 2번째와 5번째가 실제 데이타와 일치 했다고 하면, 쉽게 이렇게 [0,1,0,0,1] 표시 할 수 있습니다.
        * reciprocal_rank
            * 1/2 (0.5) # 가장 빠른 순서의 하나만을 선택 합니다
        * normalized_discounted_cumulative_gain_at_5
            * (1/log(1+2) + 1/log(1+5)) / (1/log(1+1) + 1/log(1+2)) = 0.6241
        * precision_at_5
            * 2/5 (0.4)

#### 2. 솔류션 버전의 성능 지표 분석

레서피의 종류는 크게 세가지 입니다. 각각에 대해 확인을 해보겠습니다.
#### 1. USER_PERSONALIZATION Recipes
- 5가지 (popularity,user_personalization,hrnn,hrnn_meta,hrnn_coldstart) 있습니다.
- popularity 는 베이스라인의 레서피로서 샤용을 주로 합니다. 모든 지표에서 가장 낮은 수치를 보입니다.
- hrnn, hrnn-meta, user_personalization 이 전반적으로 가장 높은 성능을 보여 줍니다. user_personalization은 default로 exploration_weight=0.3 입니다. coldstart item 이 거의 없는 상태에서도 높은 성능을 보여 주고 있습니다.학습 데이터의 인터렉션 데이터에 콜드 스타트 아이템이 전형 없기에 성능은 감소합니다. 이 경우에는 exloration_weight = 0을 하게 되면 콜드 스타트의 아이템을 배제하고 추천 합니다.<br>
이 링크 [exploration_weight](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html)에서 "Item Exploration Configuration Hyperparameters" 이 부분을 참고 하세요.<br>
- hrnn_coldstart 는coldstart item 이 거의 없는 상태이기에 성능이 낮게 나오는 것이 정상 으로 보입니다.

#### 2. RELATED_ITEMS Recipes
- sims 가 부류에 속하는 레서피로 커버리지가 높게 나왔습니다.

#### 3. PERSONALIZED_RANKING Recipes
- ranking 이 여기에 속합니다. 


<div>
    <img src="static/imgs/Fig.3.2.metric_summary2.png" width="800">
</div>

---
참고:
*추천시에 시간에 대한 입력값이 들어가기에 추천 결과가 아래 이미지 (저자 실행)와 다를 수 있습니다. 여러 번 테스트 시에 매번 조금씩 다르게 나옴을 확인 하였습니다.<br>
레서비의 정의는 개발자 문서 참고 하세요 --> [레서피 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html)


In [14]:
recipe_metrics=pd.DataFrame(metrics,columns=['recipe','coverage','mrr@25','ndcg@5','ndcg@10','ndcg@25','p@5','p@10','p@25'])

recipe_metrics

,recipe,coverage,mrr@25,ndcg@5,ndcg@10,ndcg@25,p@5,p@10,p@25
0,popularity,0.0136,0.0559,0.0395,0.0670,0.1112,0.0163,0.0200,0.0182
1,user_personalization,0.2973,0.2223,0.1820,0.2385,0.3205,0.0971,0.0874,0.0639
2,sims,0.7622,0.1802,0.1519,0.1837,0.2524,0.0689,0.0533,0.0404
3,ranking,0.2951,0.2341,0.1930,0.2454,0.3309,0.0981,0.0822,0.0622


In [15]:
%store user_personalization_solution_version_arn
%store user_personalization_solution_arn

%store sims_solution_version_arn
%store sims_solution_arn


Stored 'user_personalization_solution_version_arn' (str)
Stored 'user_personalization_solution_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'sims_solution_arn' (str)
